### 本篇代码用于建立deep learning的模型，包含evaluation

In [ ]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pickle
import json
tqdm.pandas()
pd.options.mode.chained_assignment = None
import keras
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
df = pd.read_pickle("Pickles/processed_data.pkl")
#df = pd.read_csv("Data/lemmatized_text.csv")

In [ ]:
df.describe()

In [ ]:
#df = df.reset_index()
df.index

In [ ]:
df.head()

#### 导入neural network所需package

In [2]:
import tensorflow as tf 
from keras.models import Sequential, Model, load_model 
from keras.layers import Dense, Dropout, GRU,Input, LSTM, Embedding, Bidirectional,SimpleRNN
from keras.layers import Flatten, Conv1D, MaxPooling1D, GlobalMaxPooling1D, TimeDistributed, BatchNormalization
from keras.layers import concatenate as lconcat
from keras.optimizers import SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K


#sess_config.gpu_options.allow_growth = True
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
from sklearn.metrics import roc_auc_score
from keras.utils import np_utils,plot_model, multi_gpu_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from sklearn.model_selection import StratifiedShuffleSplit #sklearn是python自带的机器学习库
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler


检查设备运算能力

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
#我只有一个CPU

#### 定义生成词向量的函数

定义函数——embedding, padding, resampling  
使用glove训练好的embedding,其中包含50，100，200，300维度。这里是假设该库可以代表当前文本。  
注意这些函数都是针对上一步的数据设计的，其中涉及对某些列的引用,容易出错

In [ ]:
df['text_len'].describe()

In [19]:
#### Define number of words, and embedding dimensions
max_words = 23070
embed_dim = 100 
#这个函数是将文本转化为embedding
def load_embeddings(vec_file):
    print("Loading Glove Model")
    #f = open(vec_file,'r')
    f = open(vec_file,'r',encoding='UTF-8') #加上encoding方式否则读取出错
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done. {} words loaded!".format(len(model)))
    return model
#这个函数是padding。
def tokenize_and_pad(docs,max_words=max_words):
    global t
    t = Tokenizer()
    t.fit_on_texts(docs)
    docs = pad_sequences(sequences = t.texts_to_sequences(docs),maxlen = max_words, padding = 'post')
    global vocab_size
    vocab_size = len(t.word_index) + 1
    
    return docs
#这个函数是调整样本的代表性
def oversample(X,docs,y):
    # Get number of rows with imbalanced class
    target = y.sum().idxmax()#得到y中数量最多的对象，up
    n = y[target].sum()#计数
    # identify imbalanced targets
    imbalanced = y.drop(target,axis=1) #扔掉up，保留另外两列
    #For each target, create a dataframe of randomly sampled rows, append to list
    #随机从另外两列中抽取一些数据进行复制
    append_list =  [y.loc[y[col]==1].sample(n=n-y[col].sum(),replace=True,random_state=20) for col in imbalanced.columns]
    append_list.append(y)
    #复制时他们的index也复制了，所以可以用来匹配X
    y = pd.concat(append_list,axis=0)
    # match y indexes on other inputs
    X = X.loc[y.index] #注意这里是根据两个表的索引来匹配的，一旦索引除了问题就会出错
    docs = pd.DataFrame(docs_train,index=y_train.index).loc[y.index]
    assert (y.index.all() == X.index.all() == docs.index.all())
    return X,docs.values,y

#### 将项目变量转换成哑变量

In [20]:
#这是把项目变量变成n-1个哑变量
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('items')),columns=mlb.classes_,),sort=False,how="left")

In [21]:
df.head()

,GICS Sector,GICS Sub Industry,cik,doc_name,release_date,text,ticker,txt_link,rm_week,rm_month,rm_qtr,rm_year,vix,price_change,signal,processed_text,text_len,sentiment,Item 1.01,Item 1.02,Item 2.01,Item 2.02,Item 2.03,Item 2.04,Item 2.05,Item 2.06,Item 3.01,Item 3.02,Item 3.03,Item 4.02,Item 5.01,Item 5.02,Item 5.03,Item 5.04,Item 5.07,Item 5.08,Item 7.01,Item 8.01,Item 9.01
0,Information Technology,Information Technology,789019.0,0001193125-19-022553.txt,2019-01-30 16:03:36,0001193125-19-022553.txt : 20190130 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,-3.80,0.61,2.12,20.47,17.66,-1.82,down,"[accession, number, conformed, submission, typ...",980,0.066667,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Information Technology,Information Technology,789019.0,0001193125-18-337951.txt,2018-11-29 15:29:57,0001193125-18-337951.txt : 20181129 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,6.77,0.11,-0.01,23.76,18.65,-0.95,stay,"[accession, number, conformed, submission, typ...",279,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Information Technology,Information Technology,789019.0,0001193125-18-306365.txt,2018-10-24 16:03:06,0001193125-18-306365.txt : 20181024 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,-0.91,-1.60,1.60,37.85,25.23,4.71,up,"[accession, number, conformed, submission, typ...",838,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Information Technology,Information Technology,789019.0,0001193125-18-277602.txt,2018-09-19 16:15:58,0001193125-18-277602.txt : 20180919 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,1.02,2.87,9.60,42.83,11.75,1.35,up,"[accession, number, conformed, submission, typ...",474,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,Information Technology,Information Technology,789019.0,0001193125-18-221458.txt,2018-07-19 16:02:35,0001193125-18-221458.txt : 20180719 0001193125...,MSFT,https://www.sec.gov/Archives/edgar/data/789019...,0.66,5.09,8.96,38.96,12.87,5.21,up,"[accession, number, conformed, submission, typ...",1332,0.333333,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


#### 将其他类别变量转换成哑变量

In [22]:
# 把label和数据分开，所有类别变量都变成哑变量
cols = ['GICS Sector','vix','rm_week','rm_month', 'rm_qtr', 'rm_year','sentiment']
cols.extend(list(mlb.classes_))
X = df[cols]
docs = df['processed_text']

y = df['signal']

# Get Dummies

docs = tokenize_and_pad(docs)
X = pd.get_dummies(columns = ['GICS Sector'],prefix="sector",data=X)
y = pd.get_dummies(columns=['signal'],data=y)

aux_shape = len(X.columns)

检查有没有出错

In [23]:
len(docs)

3426

In [24]:
len(X)

3426

In [25]:
len(y)

3426

In [26]:
X.head()

,vix,rm_week,rm_month,rm_qtr,rm_year,sentiment,Item 1.01,Item 1.02,Item 2.01,Item 2.02,Item 2.03,Item 2.04,Item 2.05,Item 2.06,Item 3.01,Item 3.02,Item 3.03,Item 4.02,Item 5.01,Item 5.02,Item 5.03,Item 5.04,Item 5.07,Item 5.08,Item 7.01,Item 8.01,Item 9.01,sector_Communication Services,sector_Consumer Discretionary,sector_Consumer Staples,sector_Energy,sector_Financials,sector_Health Care,sector_Industrials,sector_Information Technology,sector_Materials,sector_Real Estate,sector_Utilities
0,17.66,-3.80,0.61,2.12,20.47,0.066667,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,18.65,6.77,0.11,-0.01,23.76,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,25.23,-0.91,-1.60,1.60,37.85,0.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,11.75,1.02,2.87,9.60,42.83,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,12.87,0.66,5.09,8.96,38.96,0.333333,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


#### 将数据分成training, validation, testing

In [ ]:
# Split into train and test data for NN models
X_train, X_test, y_train, y_test, docs_train, docs_test = train_test_split(X, y,docs,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state = 20)
X_validation, X_test, y_validation, y_test, docs_validation, docs_test = train_test_split(X_test_va, y_test_va,docs_test_va,
                                                  stratify=y_test_va, 
                                                   test_size=0.5,
                                                    random_state = 20)

检查是否正确

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
len(docs_train)

In [ ]:
len(X_test)

#### 标准化连续变量，以及重复抽样训练数据，调整样本比例

In [ ]:
cont_features = ['vix','rm_week','rm_month', 'rm_qtr', 'rm_year','sentiment']
aux_features = cont_features + [item for item in mlb.classes_]
x_scaler = StandardScaler()
X_train[cont_features] = x_scaler.fit_transform(X_train[cont_features])
X_test[cont_features] = x_scaler.transform(X_test[cont_features])

X_train, docs_train, y_train = oversample(X_train, docs_train, y_train)

进行resample调整比重后再次检查

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
len(docs_train)

In [ ]:
y_train.sum() #每个label都变得相等

#### 用GloVe做embedding

In [ ]:
embeddings_index = load_embeddings("Data/glove.6B.100d.txt")

In [ ]:

words_not_found = []

embedding_matrix = np.zeros((vocab_size, embed_dim))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

#### 定义evaluation 函数

In [3]:
from sklearn.metrics import roc_auc_score

# 定义一个custom metric用于神经网络的评估
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.metrics.auc(y_true,y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

#### 保存数据

In [ ]:
# Save data
np.save("Pickles/docs_train.npy",docs_train)
np.save("Pickles/docs_test.npy",docs_test)
np.save("Pickles/docs_validation.npy",docs_validation)

X_train.to_pickle("Pickles/X_train.pkl")
X_test.to_pickle("Pickles/X_test.pkl")
X_validation.to_pickle("Pickles/X_validation.pkl")

y_train.to_pickle("Pickles/y_train.pkl")
y_test.to_pickle("Pickles/y_test.pkl") 
y_validation.to_pickle("Pickles/y_validation.pkl") 

np.save("Pickles/embedding_matrix.npy",embedding_matrix)

### 1.Training

测试4类神经网络模型：MLP, CNN, RNN, CNN-RNN.  
由于时间限制，不可能将每个模型的参数都调到最优。根据文献中设置参数。

In [4]:
#若要直接从下面的代码开始执行，需运行这里的系数
vocab_size = 119398
embedding_matrix = np.load("Pickles/embedding_matrix.npy")
max_words = 23070
embed_dim = 100 
aux_shape = 38

#### 定义一个函数生成不同类型的神经网络模型

In [5]:
def build_model(output_classes,architecture,aux_shape=aux_shape,vocab_size=vocab_size,embed_dim=embed_dim,embedding_matrix=embedding_matrix,max_seq_len=max_words):
    
    with tf.device('/cpu:0'): #在当前的cpu上运行
        main_input= Input(shape=(max_seq_len,),name='doc_input') #主要输入为文本，Input的维度是最长的文件max words
        main = Embedding(input_dim = vocab_size,
                            output_dim = embed_dim,
                            weights=[embedding_matrix], 
                            input_length=max_seq_len, 
                            trainable=False)(main_input)
#设置不同的模型供选择
    if architecture == 'mlp': 
        # Densely Connected Neural Network (Multi-Layer Perceptron)
        main = Dense(32, activation='relu')(main)
        main = Dropout(0.2)(main)
        main = Flatten()(main)
    elif architecture == 'cnn':
        # 1-D Convolutional Neural Network
        main = Conv1D(64, 3, strides=1, padding='same', activation='relu')(main)
        #Cuts the size of the output in half, maxing over every 2 inputs
        main = MaxPooling1D(pool_size=3)(main)
        main = Dropout(0.2)(main)
        main = Conv1D(32, 3, strides=1, padding='same', activation='relu')(main)
        main = GlobalMaxPooling1D()(main)
        #model.add(Dense(output_classes, activation='softmax'))
    elif architecture == 'rnn':
        # LSTM network
        main = SimpleRNN(32)(main)
        #main = GRU(64,activation='tanh')(main)
        #main = LSTM(32, return_sequences=False, dropout=0.2, recurrent_dropout=0.1)(main)
        #main = Bidirectional(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.1))(main)
        main = BatchNormalization()(main)
    elif architecture =="rnn_cnn":
        main = Conv1D(64, 5, padding='same', activation='relu')(main)
        main = MaxPooling1D()(main)
        main = Dropout(0.2)(main)
        #main = Bidirectional(CuDNNGRU(32,return_sequences=False),merge_mode='concat')(main)
        main = SimpleRNN(32,return_sequences=False)(main)
        main = BatchNormalization()(main)
   
    else:
        print('Error: Model type not found.')
      
    #辅助输入为其他控制变量，input维度是X的变量数
    auxiliary_input = Input(shape=(aux_shape,), name='aux_input')
    x = lconcat([main, auxiliary_input])#把两个输入合并
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    main_output = Dense(output_classes, activation='softmax', name='main_output')(x)#最终的输出是3个维度
    model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output],name=architecture)#该函数返回一个完整的模型，输入，输出，模型名
      
        #没有GPU用不了
    #sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    #model = multi_gpu_model(model)
    model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy',auc_roc])
    
    return model



#### 观察每种模型的结构

In [6]:
from keras.utils.vis_utils import plot_model
import pydot

In [7]:
mlp=build_model(3,'mlp')

In [9]:
mlp.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
doc_input (InputLayer)          (None, 23070)        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 23070, 100)   11939800    doc_input[0][0]                  
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 23070, 32)    3232        embedding_2[0][0]                
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 23070, 32)    0           dense_5[0][0]                    
__________________________________________________________________________________________________
flatten_2 

In [10]:
cnn=build_model(3,'cnn')
cnn.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
doc_input (InputLayer)          (None, 23070)        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 23070, 100)   11939800    doc_input[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 23070, 64)    19264       embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 7690, 64)     0           conv1d_1[0][0]                   
__________________________

In [11]:
rnn=build_model(3,'rnn')
rnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
doc_input (InputLayer)          (None, 23070)        0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 23070, 100)   11939800    doc_input[0][0]                  
__________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)        (None, 32)           4256        embedding_4[0][0]                
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32)           128         simple_rnn_1[0][0]               
__________________________________________________________________________________________________
aux_input 

In [12]:
cnn_rnn=build_model(3,'rnn_cnn')
cnn_rnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
doc_input (InputLayer)          (None, 23070)        0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 23070, 100)   11939800    doc_input[0][0]                  
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 23070, 64)    32064       embedding_5[0][0]                
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, 11535, 64)    0           conv1d_3[0][0]                   
__________________________________________________________________________________________________
dropout_8 

In [6]:
X_train = pd.read_pickle("Pickles/X_train.pkl")
y_train = pd.read_pickle("Pickles/y_train.pkl")
docs_train = np.load("Pickles/docs_train.npy")


#### 开始训练

#### MLP  
训练mlp到35-45epochs的时候，auc开始达到0.8，loss变得很小，0.22几

In [ ]:
#keras 2.1.0的输入必须将df变成df.values，不好用tokenizer
#keras 2.2.4不好用softmax
#mlp_20epochs = build_model(3,"mlp")

mlp_35epochs.fit([docs_train,X_train],y_train,batch_size=128,epochs=5,verbose=1)
#verbose=0为不输出日志，1为每个batch都要记录，2为每一个epoch进行记录 

mlp_35epochs.save("Data/models/mlp_45epochs.hdf5")


#### CNN

10epochs - loss:0.64,acc:0.73,auc:0.65  
13epochs - loss:0.547,acc:0.774,auc:0.873  
15epochs - loss:0.491, acc=0.799, auc=0.879  
18epochs - loss:0.4347, accu=0.8229,auc=0.901

In [ ]:
#cnn = build_model(3,"cnn")
#cnn10 = load_model("Data/models/cnn.hdf5",custom_objects={"auc_roc":auc_roc})

#model_dict["cnn"] = cnn.fit([docs_train,X_train],y_train,batch_size=64,epochs=10,verbose=1)
cnn13.fit([docs_train,X_train],y_train,batch_size=128,epochs=3,verbose=1)

cnn13.save("Data/models/cnn18.hdf5")
#with open('Data/trainHistory/cnn.pkl', 'wb') as file_pi:
 #   pickle.dump(model_dict["cnn"].history, file_pi)

#### RNN

In [9]:
rnn = build_model(3,'rnn')

5epochs: loss=1.02,accu=0.484,auc=0.6  
10epochs: loss=0.94,acc=0.54,auc=0.64  
15epochs: loss=0.896,acc=0.576,auc=0.651  
20epochs:loss=0.868,acc=0.587,auc=0.657

In [ ]:
#rnn = build_model(3,"rnn")

rnn.fit([docs_train,X_train],y_train,batch_size=128,epochs=5,verbose=1)


rnn.save("Data/models/rnn20.hdf5")
#with open('Data/trainHistory/rnn.pkl', 'wb') as file_pi:
 #   pickle.dump(model_dict["rnn"].history, file_pi)

#### RNN_CNN

In [16]:
rnn_cnn = build_model(3,"rnn_cnn")

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


5epochs: loss=0.95,acc==0.529,auc=0.645  
20epochs: loss=0.8398,acc=0.607,auc=0.702

In [ ]:
#rnn_cnn = build_model(3,"rnn_cnn")

rnn_cnn.fit([docs_train,X_train],y_train,batch_size=32,epochs=5,verbose=1)

rnn_cnn.save("Data/models/rnn_cnn5.hdf5")

rnn_cnn.fit([docs_train,X_train],y_train,batch_size=32,epochs=5,verbose=1)

rnn_cnn.save("Data/models/rnn_cnn10.hdf5")

rnn_cnn.fit([docs_train,X_train],y_train,batch_size=32,epochs=5,verbose=1)

rnn_cnn.save("Data/models/rnn_cnn15.hdf5")

rnn_cnn.fit([docs_train,X_train],y_train,batch_size=32,epochs=5,verbose=1)

rnn_cnn.save("Data/models/rnn_cnn20.hdf5")
#with open('Data/trainHistory/rnn_cnn.pkl', 'wb') as file_pi:
 #   pickle.dump(model_dict["rnn_cnn"].history, file_pi)

### 2.Validation

In [9]:
X_val = pd.read_pickle("Pickles/X_validation.pkl")
y_val = pd.read_pickle("Pickles/y_validation.pkl")
docs_val = np.load("Pickles/docs_validation.npy")


#### 2.1 validate MLP

MLP模型表现：validation的表现不如训练集，随着训练时长的增加，仿佛出现了overfitting，因为loss在val中增加，但auc和accuracy增加非常平缓。  

In [37]:
import gc

In [15]:
del mlp
gc.collect()

14170

定义一个函数让每个模型跑10遍

In [20]:
def val(model):
    score =[]
    for i in range(10):
        score.append(model.evaluate([docs_val,X_val],y_val,batch_size=128))
    score = pd.DataFrame(score)
    return score

依次将模型运行validation的数据

In [10]:
mlp = load_model("Data/models/mlp.hdf5",custom_objects={"auc_roc":auc_roc})
#mlp_20epochs = load_model("Data/models/mlp_20epochs.hdf5",custom_objects={"auc_roc":auc_roc})
#mlp_25epochs = load_model("Data/models/mlp_25epochs.hdf5",custom_objects={"auc_roc":auc_roc})
#mlp_30epochs = load_model("Data/models/mlp_30epochs.hdf5",custom_objects={"auc_roc":auc_roc})
#mlp_35epochs = load_model("Data/models/mlp_35epochs.hdf5",custom_objects={"auc_roc":auc_roc})
#mlp_40epochs = load_model("Data/models/mlp_40epochs.hdf5",custom_objects={"auc_roc":auc_roc})
#mlp_45epochs = load_model("Data/models/mlp_45epochs.hdf5",custom_objects={"auc_roc":auc_roc})


In [116]:
mlp45_score = val(mlp_45epochs)

343/343 [==============================] - ETA:  - ETA:  - 8s 23ms/step


In [44]:
#mlp的表现，平均auc=0.591,accuracy = 0.53
score.describe()

,0,1,2
count,1.000000e+01,10.000000,10.000000
mean,1.100725e+00,0.530612,0.591120
std,2.340556e-16,0.000000,0.070190
min,1.100725e+00,0.530612,0.391364
25%,1.100725e+00,0.530612,0.612871
50%,1.100725e+00,0.530612,0.613028
75%,1.100725e+00,0.530612,0.613375
max,1.100725e+00,0.530612,0.614736


In [47]:
score.to_pickle("Data//model_performance//mlp_score.pkl")

In [102]:
mlp20_score.describe()

,0,1,2
count,10.000000,10.000000,10.000000
mean,1.841277,0.548105,0.624392
std,0.000000,0.000000,0.075548
min,1.841277,0.548105,0.409380
25%,1.841277,0.548105,0.648079
50%,1.841277,0.548105,0.648152
75%,1.841277,0.548105,0.648311
max,1.841277,0.548105,0.648927


In [103]:
mlp20_score.to_pickle("Data//model_performance//mlp20_score.pkl")

In [107]:
#mlp25的表现，平均auc=0.62,accuracy = 0.517
mlp25_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,2.139504e+00,5.393586e-01,0.627250
std,4.681111e-16,1.170278e-16,0.077145
min,2.139504e+00,5.393586e-01,0.407693
25%,2.139504e+00,5.393586e-01,0.651642
50%,2.139504e+00,5.393586e-01,0.651646
75%,2.139504e+00,5.393586e-01,0.651652
max,2.139504e+00,5.393586e-01,0.651655


In [108]:
mlp25_score.to_pickle("Data//model_performance//mlp25_score.pkl")

In [81]:
#mlp30的表现，平均auc=0.63,accuracy=0.528
mlp30_score.describe()

,0,1,2
count,1.000000e+01,10.000000,10.000000
mean,2.738818e+00,0.527697,0.631081
std,4.681111e-16,0.000000,0.038684
min,2.738818e+00,0.527697,0.520984
25%,2.738818e+00,0.527697,0.643265
50%,2.738818e+00,0.527697,0.643340
75%,2.738818e+00,0.527697,0.643366
max,2.738818e+00,0.527697,0.643379


In [83]:
mlp30_score.to_pickle("Data//model_performance//mlp30_score.pkl")

In [87]:
#mlp35, auc=0.64, accuracy=0.525
mlp35_score.describe()

,0,1,2
count,1.000000e+01,10.000000,10.000000
mean,2.886647e+00,0.524781,0.646354
std,4.681111e-16,0.000000,0.037935
min,2.886647e+00,0.524781,0.538392
25%,2.886647e+00,0.524781,0.658166
50%,2.886647e+00,0.524781,0.658230
75%,2.886647e+00,0.524781,0.658371
max,2.886647e+00,0.524781,0.658947


In [88]:
mlp35_score.to_pickle("Data//model_performance//mlp35_score.pkl")

In [112]:
#mlp40, auc=0.64, accuracy=0.55
mlp40_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,2.613426e+00,5.539359e-01,0.640610
std,4.681111e-16,1.170278e-16,0.078075
min,2.613426e+00,5.539359e-01,0.418405
25%,2.613426e+00,5.539359e-01,0.665162
50%,2.613426e+00,5.539359e-01,0.665215
75%,2.613426e+00,5.539359e-01,0.665326
max,2.613426e+00,5.539359e-01,0.665711


In [113]:
mlp40_score.to_pickle("Data//model_performance//mlp40_score.pkl")

In [117]:
#mlp45, auc=0.65, accuracy=0.545
mlp45_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,2.409481e+00,5.451895e-01,0.652090
std,4.681111e-16,1.170278e-16,0.080093
min,2.409481e+00,5.451895e-01,0.424142
25%,2.409481e+00,5.451895e-01,0.677389
50%,2.409481e+00,5.451895e-01,0.677404
75%,2.409481e+00,5.451895e-01,0.677431
max,2.409481e+00,5.451895e-01,0.677473


In [118]:
mlp45_score.to_pickle("Data//model_performance//mlp45_score.pkl")

#### 2.2 validate CNN

评估cnn, 看来cnn13是最优的模型，从此之后loss开始增加，auc不再增加

In [144]:
del cnn18
gc.collect()

12283

In [140]:
cnn18 = load_model("Data/models/cnn18.hdf5",custom_objects={"auc_roc":auc_roc})
cnn18_score = val(cnn18)

343/343 [==============================] - ETA: 10 - ETA: 4 - 17s 49ms/step


In [127]:
cnn10_score.describe()

,0,1,2
count,10.000000,1.000000e+01,10.000000
mean,0.880087,6.239067e-01,0.746426
std,0.000000,1.170278e-16,0.086347
min,0.880087,6.239067e-01,0.500699
25%,0.880087,6.239067e-01,0.772913
50%,0.880087,6.239067e-01,0.773187
75%,0.880087,6.239067e-01,0.773807
max,0.880087,6.239067e-01,0.776449


In [128]:
cnn10_score.to_pickle("Data//model_performance//cnn10_score.pkl")

In [131]:
cnn13_score.describe()

,0,1,2
count,10.000000,1.000000e+01,10.000000
mean,0.797131,7.055394e-01,0.801152
std,0.000000,1.170278e-16,0.094568
min,0.797131,7.055394e-01,0.532016
25%,0.797131,7.055394e-01,0.830484
50%,0.797131,7.055394e-01,0.830675
75%,0.797131,7.055394e-01,0.831109
max,0.797131,7.055394e-01,0.832970


In [132]:
cnn13_score.to_pickle("Data//model_performance//cnn13_score.pkl")

In [137]:
cnn15_score.describe()

,0,1,2
count,1.000000e+01,10.000000,10.000000
mean,8.140949e-01,0.702624,0.807044
std,1.170278e-16,0.000000,0.095185
min,8.140949e-01,0.702624,0.536152
25%,8.140949e-01,0.702624,0.836579
50%,8.140949e-01,0.702624,0.836765
75%,8.140949e-01,0.702624,0.837191
max,8.140949e-01,0.702624,0.839042


In [138]:
cnn15_score.to_pickle("Data//model_performance//cnn15_score.pkl")

In [141]:
cnn18_score.describe()

,0,1,2
count,1.000000e+01,10.000000,10.000000
mean,8.677125e-01,0.708455,0.805866
std,1.170278e-16,0.000000,0.095470
min,8.677125e-01,0.708455,0.534159
25%,8.677125e-01,0.708455,0.835554
50%,8.677125e-01,0.708455,0.835720
75%,8.677125e-01,0.708455,0.836098
max,8.677125e-01,0.708455,0.837742


In [142]:
cnn18_score.to_pickle("Data//model_performance//cnn18_score.pkl")

#### 2.3 validate RNN

 评估RNN,看来10个epochs是最优的模型

In [36]:
del rnn20
gc.collect()

85

In [28]:
rnn20 = load_model("Data/models/rnn20.hdf5",custom_objects={"auc_roc":auc_roc})
rnn20_score = val(rnn20)

343/343 [==============================] - ETA: 20 - ETA: 8 - 34s 98ms/step


In [22]:
rnn5_score.describe()

,0,1,2
count,10.000000,1.000000e+01,10.000000
mean,1.053606,4.489796e-01,0.609112
std,0.000000,5.851389e-17,0.076528
min,1.053606,4.489796e-01,0.391316
25%,1.053606,4.489796e-01,0.632914
50%,1.053606,4.489796e-01,0.633467
75%,1.053606,4.489796e-01,0.633680
max,1.053606,4.489796e-01,0.633793


In [30]:
rnn5_score.to_pickle("Data//model_performance//rnn5_score.pkl")

In [25]:
rnn10_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,9.823108e-01,5.014577e-01,0.658322
std,1.170278e-16,1.170278e-16,0.083609
min,9.823108e-01,5.014577e-01,0.420377
25%,9.823108e-01,5.014577e-01,0.684207
50%,9.823108e-01,5.014577e-01,0.684987
75%,9.823108e-01,5.014577e-01,0.685282
max,9.823108e-01,5.014577e-01,0.685437


In [31]:
rnn10_score.to_pickle("Data//model_performance//rnn10_score.pkl")

In [27]:
rnn15_score.describe()

,0,1,2
count,10.000000,10.000000,10.000000
mean,1.049107,0.434402,0.603247
std,0.000000,0.000000,0.076260
min,1.049107,0.434402,0.386216
25%,1.049107,0.434402,0.626911
50%,1.049107,0.434402,0.627545
75%,1.049107,0.434402,0.627786
max,1.049107,0.434402,0.627912


In [32]:
rnn15_score.to_pickle("Data//model_performance//rnn15_score.pkl")

In [29]:
rnn20_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,1.015358e+00,4.285714e-01,0.614499
std,2.340556e-16,5.851389e-17,0.080089
min,1.015358e+00,4.285714e-01,0.386588
25%,1.015358e+00,4.285714e-01,0.638989
50%,1.015358e+00,4.285714e-01,0.640175
75%,1.015358e+00,4.285714e-01,0.640618
max,1.015358e+00,4.285714e-01,0.640849


In [33]:
rnn20_score.to_pickle("Data//model_performance//rnn20_score.pkl")

#### 2.4 validate CNN-RNN

评估cnn_rnn, 15个epochs最优

In [52]:
del rnn_cnn20
gc.collect()

16237

In [49]:
rnn_cnn20 = load_model("Data/models/rnn_cnn20.hdf5",custom_objects={"auc_roc":auc_roc})
rnn_cnn20_score = val(rnn_cnn20)

343/343 [==============================] - ETA: 27 - ETA: 11 - 46s 133ms/step


In [38]:
rnn_cnn5_score.describe()

,0,1,2
count,10.000000,1.000000e+01,10.000000
mean,1.119662,4.373178e-01,0.589241
std,0.000000,5.851389e-17,0.072656
min,1.119662,4.373178e-01,0.382460
25%,1.119662,4.373178e-01,0.612066
50%,1.119662,4.373178e-01,0.612267
75%,1.119662,4.373178e-01,0.612349
max,1.119662,4.373178e-01,0.612392


In [39]:
rnn_cnn5_score.to_pickle("Data//model_performance//rnn_cnn5_score.pkl")

In [42]:
rnn_cnn10_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,1.013498e+00,4.402332e-01,0.641149
std,2.340556e-16,5.851389e-17,0.079052
min,1.013498e+00,4.402332e-01,0.416165
25%,1.013498e+00,4.402332e-01,0.665998
50%,1.013498e+00,4.402332e-01,0.666198
75%,1.013498e+00,4.402332e-01,0.666279
max,1.013498e+00,4.402332e-01,0.666323


In [43]:
rnn_cnn10_score.to_pickle("Data//model_performance//rnn_cnn10_score.pkl")

In [46]:
rnn_cnn15_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,9.205074e-01,5.539359e-01,0.702369
std,1.170278e-16,1.170278e-16,0.089008
min,9.205074e-01,5.539359e-01,0.449058
25%,9.205074e-01,5.539359e-01,0.729987
50%,9.205074e-01,5.539359e-01,0.730735
75%,9.205074e-01,5.539359e-01,0.731015
max,9.205074e-01,5.539359e-01,0.731162


In [47]:
rnn_cnn15_score.to_pickle("Data//model_performance//rnn_cnn15_score.pkl")

In [50]:
rnn_cnn20_score.describe()

,0,1,2
count,10.000000,1.000000e+01,10.000000
mean,0.996781,4.693878e-01,0.658083
std,0.000000,5.851389e-17,0.081443
min,0.996781,4.693878e-01,0.426294
25%,0.996781,4.693878e-01,0.683661
50%,0.996781,4.693878e-01,0.683904
75%,0.996781,4.693878e-01,0.683999
max,0.996781,4.693878e-01,0.684050


In [51]:
rnn_cnn20_score.to_pickle("Data//model_performance//rnn_cnn20_score.pkl")

### 3. Testing

从目前来看，在训练集中cnn是表现最好的，效率也是很优秀的。以下是每类模型中的最优模型：  
mlp_20epochs  
cnn13  
rnn10  
rnn_cnn15

在测试集中，cnn表现最好，平均auc达到0.738，acc达到0.615

In [62]:
del cnn13
gc.collect()

91

In [53]:
X_test = X_test.iloc[343:]
y_test = y_test.iloc[343:]
docs_test = docs_test[343:]

定义函数让每个模型在test中跑10遍

In [54]:
def test(model):
    score =[]
    for i in range(10):
        score.append(model.evaluate([docs_test,X_test],y_test,batch_size=128))
    score = pd.DataFrame(score)
    return score

#### 3.1 test MLP

In [55]:
mlp_20epochs = load_model("Data/models/mlp_20epochs.hdf5",custom_objects={"auc_roc":auc_roc})
mlp20_test_score = test(mlp_20epochs)

343/343 [==============================] - ETA:  - ETA:  - 8s 23ms/step


In [56]:
mlp20_test_score.describe()

,0,1,2
count,1.000000e+01,10.000000,10.000000
mean,2.135054e+00,0.498542,0.605070
std,4.681111e-16,0.000000,0.077756
min,2.135054e+00,0.498542,0.383787
25%,2.135054e+00,0.498542,0.629064
50%,2.135054e+00,0.498542,0.629914
75%,2.135054e+00,0.498542,0.630229
max,2.135054e+00,0.498542,0.630393


In [57]:
mlp20_test_score.to_pickle("Data//model_performance//mlp20_test_score.pkl")

#### 3.2 test CNN

In [58]:
cnn13 = load_model("Data/models/cnn13.hdf5",custom_objects={"auc_roc":auc_roc})
cnn13_test_score = test(cnn13)

343/343 [==============================] - ETA: 14 - ETA: 5 - 23s 66ms/step


In [59]:
cnn13_test_score.describe()

,0,1,2
count,10.000000,1.000000e+01,10.000000
mean,0.965634,6.151603e-01,0.738147
std,0.000000,1.170278e-16,0.090811
min,0.965634,6.151603e-01,0.479694
25%,0.965634,6.151603e-01,0.766810
50%,0.965634,6.151603e-01,0.766887
75%,0.965634,6.151603e-01,0.766916
max,0.965634,6.151603e-01,0.766931


In [60]:
cnn13_test_score.to_pickle("Data//model_performance//cnn13_test_score.pkl")

#### 3.3 test RNN

In [63]:
rnn10 = load_model("Data/models/rnn10.hdf5",custom_objects={"auc_roc":auc_roc})
rnn10_test_score = test(rnn10)

343/343 [==============================] - ETA: 18 - ETA: 7 - 32s 94ms/step


In [64]:
rnn10_test_score.describe()

,0,1,2
count,1.000000e+01,1.000000e+01,10.000000
mean,1.006949e+00,4.548105e-01,0.642065
std,2.340556e-16,5.851389e-17,0.080912
min,1.006949e+00,4.548105e-01,0.411791
25%,1.006949e+00,4.548105e-01,0.667251
50%,1.006949e+00,4.548105e-01,0.667819
75%,1.006949e+00,4.548105e-01,0.668032
max,1.006949e+00,4.548105e-01,0.668144


In [65]:
rnn10_test_score.to_pickle("Data//model_performance//rnn10_test_score.pkl")

#### 3.4 test CNN-RNN

In [66]:
rnn_cnn15 = load_model("Data/models/rnn_cnn15.hdf5",custom_objects={"auc_roc":auc_roc})
rnn_cnn15_test_score = test(rnn_cnn15)

343/343 [==============================] - ETA: 31 - ETA: 14 - 55s 160ms/step


In [67]:
rnn_cnn15_test_score.describe()

,0,1,2
count,10.000000,1.000000e+01,10.000000
mean,0.985199,5.102041e-01,0.663844
std,0.000000,1.170278e-16,0.087919
min,0.985199,5.102041e-01,0.413666
25%,0.985199,5.102041e-01,0.690501
50%,0.985199,5.102041e-01,0.692127
75%,0.985199,5.102041e-01,0.692734
max,0.985199,5.102041e-01,0.693050


In [68]:
rnn_cnn15_test_score.to_pickle("Data//model_performance//rnn_cnn15_test_score.pkl")